In [1]:
from qiskit.circuit.random import random_circuit
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer.noise import NoiseModel
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

from math import log2
import numpy as np

In [2]:
def calculate_bias(output_map, unwanted_states):
    total_count = sum(output_map.values())

    bias = 0
    for key, val in output_map.items():
        if key in unwanted_states:
            bias += val / total_count
            
    return bias

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum", token='ce548974fc87b6f856990d2c1ca021d4c94fe6698b8b96ddb9c68bbea2880675d25b019a86b0eb832eaca814000f6fa01eb613d179fba9f9663bd41daba39689')
print(service.backends())

[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_kyiv')>, <IBMBackend('ibm_sherbrooke')>]


In [4]:
def generate_all_possible_outcomes(num_qubits):
    return {format(i, '0' + str(num_qubits) + 'b') for i in range(2**num_qubits)}

In [ ]:
backends = ["ibm_brisbane", "ibm_kyiv", "ibm_sherbrooke"]
max_qubits = 12

# Define the desired mean and median of the lognormal distribution
desired_mean = 10
desired_median = 6.5
mu = np.log(desired_median)
sigma_squared = 2 * np.log(desired_mean / desired_median)
sigma = np.sqrt(sigma_squared)

counter = 0  # Counts circuits where (ideal bias < noisy bias)
failed_results = []  # Save circuits that did not satisfy this condition
for back in backends:
    # Initialize backend and noise model (requires service, see below)
    # Example with a QiskitRuntimeService instance:
    # backend = service.backend(back)
    # For this example we assume you have a valid backend object.
    backend = service.backend(back)
    noise_model = NoiseModel.from_backend(backend)
    
    # Run 1000 circuit experiments per backend
    for _ in range(1000):
        # Generate random qubit count from a lognormal distribution within [1, max_qubits]
        while True:
            q = np.random.lognormal(mean=mu, sigma=sigma)
            round_choice = np.random.choice([0, 1])  # 0 -> floor, 1 -> ceil
            q = np.floor(q) if round_choice == 0 else np.ceil(q)
            if q >= 1 and q <= max_qubits:
                break
        qubits = int(q)
        
        # Choose a random circuit depth (most experiments use shallow circuits)
        max_depth = 5
        depth = np.random.randint(1, max_depth + 1)
        
        # Generate a random circuit with measurements
        circ = random_circuit(qubits, depth, measure=True)
        
        # Create ideal and noisy simulators
        ideal_simulator = AerSimulator()
        noisy_simulator = AerSimulator(noise_model=noise_model)
        
        # Transpile for both simulators
        circ_ideal = transpile(circ, ideal_simulator)
        circ_noisy = transpile(circ, noisy_simulator)
        
        # Run simulations (using 100000 shots for statistics)
        ideal_result = ideal_simulator.run(circ_ideal, shots=100000).result()
        ideal_dist = ideal_result.get_counts()
        noisy_result = noisy_simulator.run(circ_noisy, shots=100000).result()
        noisy_dist = noisy_result.get_counts()
        
        # Define the target state and unwanted states.
        # Here the target state is the most likely outcome in the ideal simulation.
        all_possible = generate_all_possible_outcomes(qubits)
        unwanted_states = all_possible - set(ideal_dist.keys())
        
        # Calculate bias for both distributions
        bias_ideal = calculate_bias(ideal_dist, unwanted_states)
        bias_noisy = calculate_bias(noisy_dist, unwanted_states)
        
        # We expect that noise will generally increase the bias (i.e. probability in unwanted outcomes)
        if bias_ideal < bias_noisy:
            counter += 1
        else:
            failed_results.append(circ)

print("IBM backend experiment:")
print(f"Circuits with ideal bias < noisy bias: {counter}")
print(f"Circuits that did NOT satisfy the condition: {len(failed_results)}")

In [ ]:
for circ in failed_results:
    num_qubits = circ.num_qubits
    circ_ideal = transpile(circ, ideal_simulator)
    circ_noisy = transpile(circ, noisy_simulator)
    ideal_result = ideal_simulator.run(circ_ideal, shots=100000).result()
    ideal_dist = ideal_result.get_counts()
    noisy_result = noisy_simulator.run(circ_noisy, shots=100000).result()
    noisy_dist = noisy_result.get_counts()
    target_state = max(ideal_dist, key=ideal_dist.get)
    unwanted_states = [s for s in ideal_dist if s != target_state]
    bias_ideal = calculate_bias(ideal_dist, unwanted_states)
    bias_noisy = calculate_bias(noisy_dist, unwanted_states)
    print(f"Num Qubits: {num_qubits}; Bias Ideal: {bias_ideal:.4f}; Bias Noisy: {bias_noisy:.4f}")


Num Qubits: 4; Bias Ideal: 0.6095; Bias Noisy: 0.5801
Num Qubits: 2; Bias Ideal: 0.0000; Bias Noisy: 0.0000
Num Qubits: 2; Bias Ideal: 0.0000; Bias Noisy: 0.0000
Num Qubits: 3; Bias Ideal: 0.2477; Bias Noisy: 0.2666
Num Qubits: 8; Bias Ideal: 0.0000; Bias Noisy: 0.0000
Num Qubits: 6; Bias Ideal: 0.4982; Bias Noisy: 0.0843
Num Qubits: 4; Bias Ideal: 0.4972; Bias Noisy: 0.4280
Num Qubits: 12; Bias Ideal: 0.0000; Bias Noisy: 0.0000
Num Qubits: 3; Bias Ideal: 0.4998; Bias Noisy: 0.4753
Num Qubits: 2; Bias Ideal: 0.4998; Bias Noisy: 0.4972
Num Qubits: 8; Bias Ideal: 0.8785; Bias Noisy: 0.3652
Num Qubits: 5; Bias Ideal: 0.6680; Bias Noisy: 0.6089
Num Qubits: 8; Bias Ideal: 0.0000; Bias Noisy: 0.0000
Num Qubits: 5; Bias Ideal: 0.0000; Bias Noisy: 0.0000
Num Qubits: 5; Bias Ideal: 0.0000; Bias Noisy: 0.0000
Num Qubits: 6; Bias Ideal: 0.2789; Bias Noisy: 0.0965
Num Qubits: 3; Bias Ideal: 0.5123; Bias Noisy: 0.5079
Num Qubits: 2; Bias Ideal: 0.5000; Bias Noisy: 0.4781
Num Qubits: 10; Bias Ideal:

KeyboardInterrupt: 